In [1]:
import numpy as np
import ckwrap
from normals_init import find_neighs

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [28]:
all_pts = np.load("xyz_points.npy")
all_norms = np.load("init_normals.npy")

In [31]:
'''creating patches via clustering'''

def process_one_pcd(pts, norms):
    dists_ori, indices_ori = find_neighs(pts)
    neighs = pts[indices_ori]
    print("neighs shape: ", neighs.shape)
    labels_per_row = np.apply_along_axis(lambda row: ckwrap.ckmeans(row, 5).labels, 1, dists_ori)
    print("labels shape: ",labels_per_row.shape)
    for i,( ind, labels) in enumerate(zip(indices_ori, labels_per_row)):
        for clus in np.unique(labels_per_row[0]):
            ind_clus = ind[np.where(labels == clus)]
            print("ind_clus shape: ",ind_clus.shape)
            coords = pts[ind_clus]
            crs_pdts = np.cross(pts[i], coords)
            print("crs_pdts shape: ",crs_pdts.shape)
            orient = np.dot(crs_pdts, norms[i]) #check done, works as intented
            print("orient shape: ",orient.shape)
            
            # return orient, crs_pdts
            break
        break


l,c = process_one_pcd(all_pts[0], all_norms[0])

neighs shape:  (100000, 250, 3)
labels shape:  (100000, 250)
ind_clus shape:  (24,)
crs_pdts shape:  (24, 3)
orient shape:  (24,)


In [32]:
l

array([ 0.        ,  0.0402975 , -0.08238447, -0.21255644,  0.1723234 ,
        0.2148106 ,  0.16494621, -0.23875831, -0.05734563, -0.09128525,
        0.18319691,  0.35292886,  0.08212659,  0.4166241 ,  0.06918626,
        0.29771291,  0.36442621, -0.45911662,  0.32531254, -0.44918388,
       -0.401329  ,  0.53280052,  0.32271438,  0.30455929])

In [38]:
np.dot(c[20], all_norms[0][0])

-0.4013289978202006

In [6]:
'''sanity checks'''

x = np.array([[10,9,8,7,6],[5,4,3,2,1]])
y = np.array([[0,0,1,2,0],[2,1,2,1,0]])
print(x.shape, y.shape)
# np.where(y==0, axis=1)

(2, 5) (2, 5)


In [25]:
unique_labels = np.unique(y)
label_indices = np.arange(len(y[0]))[:, None]
print(label_indices)
masks = y == 1
print(masks)
#get the x points for each label while maintaining the dimensionality
x0 = np.where(masks, x, np.nan).reshape(x.shape)  
x0 = x0[~np.all(np.isnan(x0), axis=1)].astype(object)

print(x0)
#drop nan from x0
# x0 = x0[~np.isnan(x0)]
# x0

[[0]
 [1]
 [2]
 [3]
 [4]]
[[False False  True False False]
 [False  True False  True False]]


IndexError: boolean index did not match indexed array along dimension 0; dimension is 2 but corresponding boolean dimension is 5